In [29]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

import spacy

device = torch.device("cuda")

In [30]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-it-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-it-en")

it_nlp = spacy.load("it_core_news_sm")
en_nlp = spacy.load("en_core_web_sm")

In [31]:
from torchinfo import summary 

print(summary(model))

Layer (type:depth-idx)                                       Param #
MarianMTModel                                                --
├─MarianModel: 1-1                                           --
│    └─Embedding: 2-1                                        41,154,048
│    └─MarianEncoder: 2-2                                    41,154,048
│    │    └─Embedding: 3-1                                   (recursive)
│    │    └─MarianSinusoidalPositionalEmbedding: 3-2         (262,144)
│    │    └─ModuleList: 3-3                                  18,914,304
│    └─MarianDecoder: 2-3                                    41,154,048
│    │    └─Embedding: 3-4                                   (recursive)
│    │    └─MarianSinusoidalPositionalEmbedding: 3-5         (262,144)
│    │    └─ModuleList: 3-6                                  25,224,192
├─Linear: 1-2                                                41,154,048
Total params: 209,278,976
Trainable params: 208,754,688
Non-trainable params: 524,2

In [120]:
def translate(text, model, tokenizer, extract_pos=False):
    # Split input text into smaller bits to be able to translate longer texts 
    doc = it_nlp(text)
    
    translations = []
    for s in doc.sents:
        if len(s) == 1:
            continue

        # Tokenize the source text
        inputs = tokenizer.encode(str(s), return_tensors="pt")

        # Perform the translation and decode the output
        outputs = model.generate(inputs, max_length=40, num_beams=4, early_stopping=True)
        translated_text = tokenizer.decode(outputs[0])
        translations.append(translated_text) 

    full_translation = "\n".join(translations)
    if extract_pos:
        noun_pairs, verb_pairs = translate_pos(doc)
        pos_output = "\nNouns:\n" + "\n".join([f"| {noun} -> {translation}" for noun, translation in noun_pairs.items()])
        pos_output += "\n\nVerbs:\n" + "\n".join([f"| {verb} -> {translation}" for verb, translation in verb_pairs.items()])
        full_translation += "\n\n" + pos_output

    return full_translation.replace("<pad>", "").replace("</s>", "")

def translate_words(words, model, tokenizer):
    # Tokenize the source words
    inputs = tokenizer.batch_encode_plus(words, return_tensors="pt", padding=True)

    # Perform the translation and decode the output
    outputs = model.generate(**inputs, max_length=40, num_beams=4, early_stopping=True)
    translated_words = [tokenizer.decode(output) for output in outputs]

    return translated_words

def translate_pos(spacy_doc):
    nouns = [token.text for token in spacy_doc if token.pos_ == "NOUN"]
    verbs = [token.text for token in spacy_doc if token.pos_ == "VERB"]

    # Translate each unique noun and verb
    unique_nouns = list(set(nouns))
    unique_verbs = list(set(verbs))
    noun_translations = translate_words(unique_nouns, model, tokenizer)
    verb_translations = translate_words(unique_verbs, model, tokenizer)

    # Create dictionaries mapping the original words to their translations
    noun_pairs = dict(zip(unique_nouns, noun_translations))
    verb_pairs = dict(zip(unique_verbs, verb_translations))

    # Replace the original words with their translations
    translated_nouns = [noun_pairs[noun] for noun in nouns]
    translated_verbs = [verb_pairs[verb] for verb in verbs]

    return dict(zip(nouns, translated_nouns)), dict(zip(verbs, translated_verbs))

text = """
LONDRA - Il pericolo di un Armageddon nucleare durante la guerra in Ucraina è apparso più vicino di quanto era trapelato ufficialmente fino ad ora. 
Nell'ottobre 2022, mentre la controffensiva di Kiev aveva raggiunto considerevoli risultati e sembrava in grado di riprendere anche la Crimea, gli Stati Uniti ottennero informazioni di intelligence secondo cui la Russia discuteva concretamente l'ipotesi di usare un'arma atomica tattica contro le forze ucraine.
E Washington considerava con crescente preoccupazione in che modo avrebbe reagito dal punto di vista militare.
"""

translated_text = translate(text, model, tokenizer, extract_pos=True)
print(translated_text)

 LONDON - The danger of a nuclear Armageddon during the war in Ukraine appeared closer than it had officially leaked until now.
 In October 2022, while Kiev's counter-offensive had achieved considerable results and seemed able to resume Crimea as well, the United States obtained intelligence information that Russia was specifically discussing the
 And Washington was increasingly concerned about how he would react from a military standpoint.


Nouns:
| pericolo ->  hazard  
| guerra ->  war  
| ottobre ->  October  
| controffensiva ->  counteroffensive
| considerevoli ->  substantial  
| risultati ->  results  
| grado ->  grade  
| informazioni ->  information  
| intelligence ->  intelligence  
| ipotesi ->  assumptions  
| arma ->  weapon  
| tattica ->  tactics 
| forze ->  forces  
| preoccupazione ->  concern  
| modo ->  mode  
| punto ->  point  
| vista ->  view  

Verbs:
| apparso ->  appeared  
| trapelato ->  trapelate
| raggiunto ->  reached  
| sembrava ->  It looked like

In [ ]:
import gradio as gr
import os
import time

# Chatbot demo with multimodal input (text, markdown, LaTeX, code blocks, image, audio, & video). Plus shows support for streaming text.


def print_like_dislike(x: gr.LikeData):
    print(x.index, x.value, x.liked)


def add_text(history, text):
    history = history + [(text, None)]
    return history, gr.Textbox(value="", interactive=False)


def add_file(history, file):
    history = history + [((file.name,), None)]
    return history


def bot(history):
    response = "**That's cool!**"
    history[-1][1] = ""
    for character in response:
        history[-1][1] += character
        time.sleep(0.05)
        yield history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        [],
        elem_id="chatbot",
        bubble_full_width=False,
        avatar_images=(None, (os.path.join(os.path.dirname(__file__), "avatar.png"))),
    )

    with gr.Row():
        txt = gr.Textbox(
            scale=4,
            show_label=False,
            placeholder="Enter text and press enter, or upload an image",
            container=False,
        )
        btn = gr.UploadButton("📁", file_types=["image", "video", "audio"])

    txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(
        bot, chatbot, chatbot, api_name="bot_response"
    )
    txt_msg.then(lambda: gr.Textbox(interactive=True), None, [txt], queue=False)
    file_msg = btn.upload(add_file, [chatbot, btn], [chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

    chatbot.like(print_like_dislike, None, None)


demo.queue()
if __name__ == "__main__":
    demo.launch()